In [457]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tqdm import tqdm
from time import sleep
from tqdm import trange
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import math as mt
from numpy.linalg import inv, det
from matplotlib import pyplot as plt
%matplotlib inline

## Read data

In [458]:
mfeat = pd.read_csv("mfeat/mfeat-fac", delimiter="\0", sep="\0", header=None )
mfeatFou = pd.read_csv("mfeat/mfeat-fou", delimiter="\0", sep="\0", header=None )
mfeatka = pd.read_csv("mfeat/mfeat-kar", delimiter="\0", sep="\0", header=None )

y = [0 for x in range(0, 200)]
y += [1 for x in range(200, 400)]
y += [2 for x in range(400, 600)]
y += [3 for x in range(600, 800)]
y += [4 for x in range(800, 1000)]
y += [5 for x in range(1000, 1200)]
y += [6 for x in range(1200, 1400)]
y += [7 for x in range(1400, 1600)]
y += [8 for x in range(1600, 1800)]
y += [9 for x in range(1800, 2000)]

mfeat

### Data preprocessing

In [459]:
#Retirando os espaços. Transformando da elemento em uma lista com valores númericos, Atribuindo isso a uma 
# lista com tudo processado e normaliza
def preprocessing(data):
    new_data = []
    for ex in data:
        ex = ex[0].split(" ")
        exemplos1 = []
        for element in ex:
            if element != "":
                exemplos1.append(element)
        new_data.append(exemplos1) 
    
    ## Normaliza
  
    new_data = np.array(new_data).astype(float)
    scaler = StandardScaler()
    scaler.fit(new_data)
    new_data = scaler.transform(new_data)
    return new_data
   


In [460]:
mfeat =  preprocessing(mfeat.values)
mfeatFou =  preprocessing(mfeatFou.values)
mfeatKa =  preprocessing(mfeatka.values)

# Dataset (Train and test)

In [461]:
X_train_mfeat, X_test_mfeat, y_train_mfeat, y_test_mefeat =  train_test_split(mfeat,y, test_size=0.10, random_state=42)
X_train_mfeatFou, X_test_mfeatFou, y_train_mfeatFou, y_test_mefeatFou =  train_test_split(mfeatFou,y, test_size=0.10, random_state=42)
X_train_mfeatKa, X_test_mfeatKa, y_train_mfeatKa, y_test_mefeatKa =  train_test_split(mfeatKa,y, test_size=0.10, random_state=42)

# Euclidean matrix 

In [462]:
matrix_euclian_mfeat = squareform(pdist(mfeat, metric='euclidean'))
matrix_euclian_mfeatFou = squareform(pdist(mfeatFou, metric='euclidean'))
matrix_euclian_mfeatKa = squareform(pdist(mfeatKa, metric='euclidean'))

# Bayes

In [463]:
def calc_density(x, priori, mean, sigma, classes):
    apriori = priori
    mean = mean
    sigma = sigma
    classes = classes
    x = x
    densities = []

    for c in range(0,classes):


        ##determinant
        inver = inv(np.identity(x.shape[0]) * sigma[c])
        determinant= det(inver)

        part_one_equation = mt.pow(2*mt.pi, -x.shape[0]/2)*mt.pow(determinant,0.5)

        ## values for exp calculation
        value1_exp = ((x -mean[c]).T)
        value1_exp = np.dot(value1_exp, inver)
      
        value2_exp = (x-mean[c])

        ##calc  exp
        exp = np.exp(-0.5*(np.dot(value1_exp, value2_exp)))

        #Result conditional x priori

        rest = part_one_equation*exp

        densities.append(rest*priori[c])
        

    return densities

In [464]:

def densities_views():
    nb = GaussianNB()
    nb.fit(X_train_mfeat, y_train_mfeat);


    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_
 

    densities_mfeat= []
    for xi in X_train_mfeat:
        densities_mfeat.append(calc_density(xi, priori, mean, sigma, 10))


    nb = GaussianNB()
    nb.fit(X_train_mfeatFou, y_train_mfeatFou);


    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_


    densities_mfeatFou= []
    for xi in X_train_mfeatFou:
        densities_mfeatFou.append(calc_density(xi, priori, mean, sigma, 10))



    nb = GaussianNB()
    nb.fit(X_train_mfeatKa, y_train_mfeatKa);
    priori = nb.class_prior_
    sigma = nb.sigma_
    mean = nb.theta_
    classes = nb.classes_


    densities_mfeatKa= []
    for xi in X_train_mfeatKa:
        densities_mfeatKa.append(calc_density(xi, priori, mean, sigma, 10))
        
        
    return densities_mfeat,densities_mfeatFou, densities_mfeatKa
    
    

In [465]:
densities_view1, densities_view2, densities_view3 = densities_views()

In [439]:
np.array(densities_view3).shape

(1800, 10)

In [440]:
def calc_evidence(densities_view1, densities_view2, densities_view3):
    evidence = np.array(densities_view1).sum() + np.array(densities_view2).sum()+ np.array(densities_view3).sum()

    return evidence

In [441]:
evidence =  calc_evidence(densities_view1, densities_view2, densities_view3)

In [442]:
def posteriori(view, evidence):
    
    view = np.array(view)
    
    return  np.array(view/evidence)

In [443]:
def predict(posteriori):
    
   
    y_pred = [np.argmax(x) for x in posteriori]
    return y_pred    

In [444]:
posteriori_view1 = posteriori(densities_view1, evidence)
posteriori_view2 = posteriori(densities_view2, evidence)
posteriori_view3 = posteriori(densities_view3, evidence)

In [445]:
posteriori_final = (posteriori_view1+posteriori_view2+posteriori_view3)

In [401]:
y_predict_view1 = predict(posteriori_view1)
y_predict_view2 = predict(posteriori_view2)
y_predict_view3 = predict(posteriori_view3)

y_pred_final = predict(posteriori_final)

In [402]:
len(y_predict_view1)

1800

In [446]:
from sklearn.metrics import accuracy_score

In [450]:
acc_view1 = accuracy_score(y_train_mfeat, y_predict_view1)
acc_view1

0.9288888888888889

In [451]:
acc_view2 = accuracy_score(y_train_mfeatFou, y_predict_view2)
acc_view2

0.7977777777777778

In [453]:
acc_view3 = accuracy_score(y_train_mfeatKa,  y_predict_view3)
acc_view3

0.9477777777777778

In [456]:
acc_final = accuracy_score(y_train_mfeatKa,  y_pred_final)

In [397]:
acc_final

0.9505555555555556